In [58]:
#this file formats the edm raw data file 

In [59]:
import pandas as pd
import numpy as np

In [60]:
df1 = pd.read_excel(
     '../original_data/new/ECTAS_EDM_vitals_Jan19_Aug20_expanded.xlsx',
     engine='openpyxl', sheet_name = 'edm_vitals'
)

In [61]:
df1

,pt_identifier,screen,date,time,n_query,result
0,H1000105,VITALS,2019-03-11,1623,N.000280.1,111
1,H1000105,VITALS,2019-03-11,1623,N.000320.1,100
2,H1000105,VITALS,2019-07-07,550,N.000260.1,36.7
3,H1000105,VITALS,2019-07-07,550,N.000280.1,86
4,H1000105,VITALS,2019-07-07,550,N.000290.1,18
...,...,...,...,...,...,...
234548,Z999621,VITALS,2020-06-05,1243,N.000320.1,100
234549,Z999621,VITALS,2020-06-05,1400,N.000280.1,72
234550,Z999621,VITALS,2020-06-05,1400,N.000290.1,16
234551,Z999621,VITALS,2020-06-05,1400,N.000300.1,122/59


In [62]:
del df1['screen']


In [63]:
rows = len(df1.index)
df1

,pt_identifier,date,time,n_query,result
0,H1000105,2019-03-11,1623,N.000280.1,111
1,H1000105,2019-03-11,1623,N.000320.1,100
2,H1000105,2019-07-07,550,N.000260.1,36.7
3,H1000105,2019-07-07,550,N.000280.1,86
4,H1000105,2019-07-07,550,N.000290.1,18
...,...,...,...,...,...
234548,Z999621,2020-06-05,1243,N.000320.1,100
234549,Z999621,2020-06-05,1400,N.000280.1,72
234550,Z999621,2020-06-05,1400,N.000290.1,16
234551,Z999621,2020-06-05,1400,N.000300.1,122/59


In [64]:
df2 = pd.DataFrame(data=[], columns= ['pt_identifier', 'Date', 'Time', 'BP1', 'BP2', 'Pulse', 'Temp', 'RR', 'O2Sat'])

In [65]:
df2

,pt_identifier,Date,Time,BP1,BP2,Pulse,Temp,RR,O2Sat


In [66]:
def count_consecutive(df1, i, last_idx):
    count = 1
    
    while np.sum(df1.iloc[i][0:3] == df1.iloc[i + 1][0:3]) == 3:
        count += 1
        i += 1
        if i == last_idx:
            break
    
    return count

In [67]:
def find_index(key_list, key):
    idx = -1
    for x in range(len(key_list)):
        if key_list[x] == key:
            idx = x
        
    return idx

In [68]:
def extract_vitals(df1, i, consecutive):
    p_id = df1.iloc[i][0]
    date = df1.iloc[i][1]
    time = df1.iloc[i][2]
       
    key_list  = list(df1.iloc[i:i+consecutive,3])
    value_list = list(df1.iloc[i:i+consecutive,4])
    
    bp_idx = find_index(key_list, 'N.000300.1')
    pulse_idx = find_index(key_list, 'N.000280.1')
    temp_idx = find_index(key_list, 'N.000260.1')
    rr_idx = find_index(key_list, 'N.000290.1')
    o2sat_idx = find_index(key_list, 'N.000320.1')

    
    bp_combined = value_list[bp_idx] if bp_idx != -1 and isinstance(value_list[bp_idx], str) else ''
    # print( type(bp_combined))
    # print(bp_combined)
    bp1, bp2 = bp_combined.split('/') if bp_combined != '' else ['','']
    
    pulse = value_list[pulse_idx] if pulse_idx != -1 else ''
    temp = value_list[temp_idx] if temp_idx != -1 else ''
    rr = value_list[rr_idx] if rr_idx != -1 else ''
    o2sat = value_list[o2sat_idx] if o2sat_idx != -1 else ''

    
    p_vitals = [p_id, date, time, bp1, bp2, pulse, temp, rr, o2sat]
    return p_vitals

In [69]:
i = 0
while i < rows:
    consecutive = count_consecutive(df1, i, rows - 1)
    p_vitals = extract_vitals(df1, i, consecutive)
    df2 = df2.append({'pt_identifier': p_vitals[0], 'Date': p_vitals[1],
                      'Time': p_vitals[2], 'BP1': p_vitals[3], 'BP2': p_vitals[4],
                      'Pulse': p_vitals[5], 'Temp': p_vitals[6], 'RR': p_vitals[7],
                      'O2Sat': p_vitals[8]}, ignore_index=True)
    i += consecutive
  
    


In [70]:
df2

,pt_identifier,Date,Time,BP1,BP2,Pulse,Temp,RR,O2Sat
0,H1000105,2019-03-11,1623,,,111,,,100
1,H1000105,2019-07-07,550,125,96,86,36.7,18,98
2,H1000105,2019-07-07,1121,,,66,36.5,16,100
3,H1000105,2019-10-25,2053,106,70,78,,16,96
4,H1000105,2019-10-26,832,115,73,60,36.4,16,99
...,...,...,...,...,...,...,...,...,...
56113,Z999621,2020-06-05,859,128,68,54,,18,97
56114,Z999621,2020-06-05,1035,147,101,64,,18,95
56115,Z999621,2020-06-05,1130,139,69,61,,16,93
56116,Z999621,2020-06-05,1243,115,40,67,,16,100


In [71]:
df2.to_csv('../FormattedData/1_expanded.csv') #formatted edm vitals


In [134]:
# run time approx 5 minutes for 75,000 rows